demo

In [1]:
d = {
    "query": "I ordered my card but it still isn't here",
    "pos": ["When will my card arrive?"],
    "neg": [
        "How do I change my PIN?",
        "How do I cancel a transfer?",
        "How do I receive money?",
        "What is the timing for transfers?",
        "Why is my virtual card not working?",
        "How do I get a physical card?",
        "Why is there an extra charge on my statement?",
        "Why is my transfer pending?",
        "How do I top up my account by card?",
        "What are the top-up limits?",
        "Why was my top-up reverted?",
        "What are the limits for disposable cards?",
        "Why is my card payment pending?",
        "How do I verify my identity?",
        "Why did I receive the wrong amount of cash?",
        "Is my card a Visa or Mastercard?",
        "How do I terminate my account?",
        "How do I get a virtual card?",
        "Why is my cash withdrawal pending?",
        "Why is my direct debit payment not recognised?",
        "Why was my transaction charged twice?",
        "Why did my top-up fail?",
        "Why is my card not working?",
        "Why is my cash withdrawal not recognised?",
        "How do I verify the source of my funds?",
        "How do I order a physical card?",
        "Which cards and currencies are supported?",
        "Why was a fee charged for my transfer?",
        "Why was my transfer declined?",
        "How do I activate my card?",
    ],
    "prompt": "",
    "type": "symmetric_class",
}

In [2]:
len(d["neg"])

30

In [1]:
# 尝试加载一下数据集，看看数据集格式
from datasets import load_dataset
dataset = load_dataset("fancyzhx/ag_news")
dataset

/opt/anaconda3/envs/embed/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/anaconda3/envs/embed/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [2]:
all_cls_d = {}
train_dataset = dataset["train"]

for item in train_dataset:
    text = item["text"]
    label = item["label"]
    if label not in all_cls_d.keys():
        all_cls_d[label] = []
    all_cls_d[label].append(text)

In [6]:
{k: len(v) for k, v in all_cls_d.items()}

{2: 30000, 3: 30000, 1: 30000, 0: 30000}

In [3]:
import random

In [8]:
random.sample(all_cls_d[0][:3], 2)

['S.Koreans Clash with Police on Iraq Troop Dispatch (Reuters) Reuters - South Korean police used water cannon in\\central Seoul Sunday to disperse at least 7,000 protesters\\urging the government to reverse a controversial decision to\\send more troops to Iraq.',
 "Palestinians in Israeli Jails Start Hunger Strike (Reuters) Reuters - Thousands of Palestinian\\prisoners in Israeli jails began a hunger strike for better\\conditions Sunday, but Israel's security minister said he\\didn't care if they starved to death."]

In [9]:
train_dataset[0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [4]:
def get_per_cls_n(n):
    def _func(item):
        item["query"] = item["text"]
        label = item["label"]
        item["pos"] = random.sample(all_cls_d[label], 1)
        item["neg"] = []
        for k, v in all_cls_d.items():
            if k == label:
                continue
            item["neg"].extend(random.sample(v, n))
        item["prompt"] = ""
        item["type"] = "symmetric_class"
        return item
    return _func

In [5]:
new_train_dataset = train_dataset.map(get_per_cls_n(10))

In [6]:
new_train_dataset[0].keys()

dict_keys(['text', 'label', 'query', 'pos', 'neg', 'prompt', 'type'])

In [7]:
new_train_dataset = new_train_dataset.remove_columns(["text", "label"])

In [8]:
new_train_dataset = new_train_dataset.select(range(500))

In [9]:
new_train_dataset

Dataset({
    features: ['query', 'pos', 'neg', 'prompt', 'type'],
    num_rows: 500
})

In [10]:
new_train_dataset.to_json("data/classification-no_in_batch_neg/ag_news_train.json")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 34.81ba/s]


3878625